## Prompt and actions on Questions


* Look at `registrants` Table in postgres
* Fetch the region/neignourhood in the query.
* Get all the block groups under the neigbourhood using neigbbourhood-block group mapping file created using overlay layer between them.
* Create bounding box extent with the region as centre of San Diego County.
* Highlight the block groups relevant for our region.
* Classify the block groups according to a score or rank created using combination of variables.

In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection, GeoAccessor
from arcgis.geocoding import geocode
from arcgis.geometry import buffer, Point

import pandas as pd

import sys
sys.path.append('../../')
from utils import get_config

In [2]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)

### Check if the user belongs to a Food desert

#### Check meal_gap or CalFresh data to check Food insecurity rate for some radius of area
* Map the meal gap data is at County level, not sure if it is really helpful for us.
* CalFresh Data might need to be standardized, but it is on zip level.

##### Can we get list of block groups given a region. For now we will consider planned community areas.

In [3]:
## Get the relevant block groups
region_of_interest = 'RANCHO PENASQUITOS'
# region_of_interest = 'MIRA MESA'
region_of_interest = region_of_interest.lower()
cpa_bg_map = pd.read_csv("../resources/san_diego_county_cpa_blk_grp_map.csv", dtype ='str')
mask = cpa_bg_map['CPANAME']==region_of_interest
block_groups_in_region = cpa_bg_map[mask]
block_groups_in_region_list = block_groups_in_region['fips'].tolist()

In [4]:
block_groups_in_region_list[:10]

['060730083361',
 '060730083362',
 '060730083371',
 '060730083372',
 '060730083373',
 '060730083651',
 '060730083652',
 '060730083661',
 '060730083662',
 '060730083663']

#### Picking some random variable from demographics for Map rending POC

In [5]:
# Query the block group for respective variables and feature layer.
variables = "demographics"
if variables=="demographics":
    url = "https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/aff1b6/FeatureServer/0"
ft_lyr = FeatureLayer(gis=gis,
                       url=url)

# Picking up sample variables for just poc on rendering maps as per our need.
out_fields = ["fips", "occeduc_cy","pif60_cy","OCCFOOD_CY"]  

#### Feature layer Query to get the features fro identified block groups.

In [6]:
where = "fips IN (" + ",".join(block_groups_in_region_list) + ")"
#print(where)
result_rows = ft_lyr.query(
    where=where,
    out_fields=out_fields,
    return_geometry=True,
    as_df=True,
)
result_rows.head(2)

,FID,fips,occeduc_cy,pif60_cy,occfood_cy,SHAPE
0,508,060730170181,41.0,2.0,82.0,"{""rings"": [[[-13035625.351, 3890389.58830002],..."
1,509,060730170182,28.0,0.0,77.0,"{""rings"": [[[-13035887.7309999, 3888939.776800..."


#### Creating a bounding box

In [7]:
map_centre = region_of_interest+", San Diego County"
print(f"Map Centre: {map_centre}")
m1 = gis.map(map_centre, zoomlevel=12)
m1

Map Centre: rancho penasquitos, San Diego County


MapView(layout=Layout(height='400px', width='100%'))

#### Identify the relevant block groups

In [8]:
map_centre = region_of_interest+", San Diego County"
print(f"Map Centre: {map_centre}")
m2 = gis.map(map_centre, zoomlevel=12)
result_rows.spatial.plot(renderer_type='c',                          
                         map_widget= m2,)
m2

Map Centre: rancho penasquitos, San Diego County


MapView(layout=Layout(height='400px', width='100%'))

#### Classify the Block Groups

In [9]:
map_centre = region_of_interest+", San Diego County"
print(f"Map Centre: {map_centre}")
m3 = gis.map(map_centre, zoomlevel=12)
result_rows.spatial.plot(renderer_type='c',
                         method='esriClassifyNaturalBreaks',
                         class_count=10,  # choose the number of classes
                         col='occeduc_cy',
                         cmap='hot',  # color map to pick colors from for each class
                         alpha=0.7 ,
                          
                         map_widget= m3,)
m3

Map Centre: rancho penasquitos, San Diego County


MapView(layout=Layout(height='400px', width='100%'))